In [ ]:
import numpy as np
import pyvista as pv
from kale.algorithms import contour_banded
from kale import theme
import xarray as xr

%config InlineBackend.figure_format = "retina"

# Load Data

In [ ]:
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.025
TIME_STEP_INTERESTING = 1000

In [ ]:
ds = xr.open_dataset(MESH_VALUES_FILE_NAME, engine="netcdf4")
# data = ds["loading_rate"][0, :]
data = ds["cumulative_slip"][40000, :]
mesh = pv.read(MESH_GEOMETRY_FILE_NAME)
mesh.points[:, -1] *= MESH_Z_SCALE

# Extract boundary of mesh
boundary = mesh.extract_feature_edges(
    boundary_edges=True,
    non_manifold_edges=False,
    feature_edges=False,
    manifold_edges=False,
)
boundary.clear_data()

# # Load fields from .npz file
# npz_data = np.load("skies_single_step_erosion_figures_nankai.npz")
# mesh.cell_data["pre_event_slip_deficit"] = npz_data["name1"]
# mesh.cell_data["event_slip"] = npz_data["name2"]
# mesh.cell_data["post_event_slip_deficit"] = npz_data["name3"]
# mesh.cell_data["meshes_areas"] = npz_data["name4"]

# Direct linear mapping

In [ ]:
mesh.cell_data["data"] = ds["cumulative_slip"][40000, :]

pl = pv.Plotter()
pl.add_mesh(boundary)

box = pv.Box(mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")

CONTOUR_LEVELS = np.linspace(0, 20, 10)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]
contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="data",
)

pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars="data",
    n_colors=N_COLORS,
    scalar_bar_args=dict(title=f"", **theme.SCALAR_BAR_OPTS),
)
pl.add_mesh(edges)
pl.show()

# Plot of same data but with log scaling

In [ ]:
mesh.cell_data["data"] = np.log10(ds["cumulative_slip"][40000, :] + 0.0001)

pl = pv.Plotter()
pl.add_mesh(boundary)

box = pv.Box(mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")

CONTOUR_LEVELS = np.linspace(0, 20, 10)
CONTOUR_LEVELS = np.logspace(-1, 0, 10)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]
contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="data",
)
pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars="data",
    n_colors=N_COLORS,
    log_scale=True,
    scalar_bar_args=dict(title=f"", **theme.SCALAR_BAR_OPTS),
)
pl.add_mesh(edges)
pl.show()